In [1]:
# general imports
from pathlib import Path

# TARDIS imports for the gamma-ray code
from tardis.io.atom_data import AtomData
from tardis.model import SimulationState
from tardis.plasma.base import BasePlasma
from tardis.io.configuration import config_reader
from tardis.io.atom_data.util import download_atom_data
from tardis.energy_input.energy_source import get_nuclear_lines_database
from tardis.energy_input.gamma_ray_channel import create_isotope_dicts, create_inventories_dict, calculate_total_decays, create_isotope_decay_df
from tardis.plasma.properties import Density, Abundance, IsotopeAbundance, \
    IsotopeNumberDensity, AtomicData, AtomicMass, IsotopeMass, NumberDensity, \
    SelectedAtoms

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
# Download the atom data file from tardis-refdata repo to run this cell.
download_atom_data('kurucz_cd23_chianti_H_He')
atom_data_file = 'kurucz_cd23_chianti_H_He.h5'
atom_data = AtomData.from_hdf(atom_data_file)

INFO:tardis.io.atom_data.base:Reading Atom Data with: UUID = 6f7b09e887a311e7a06b246e96350010 MD5  = 864f1753714343c41f99cb065710cace 
INFO:tardis.io.atom_data.base:Non provided Atomic Data: synpp_refs, photoionization_data, yg_data, two_photon_data, linelist


In [3]:
# Read the config file and create a configuration object
config = config_reader.Configuration.from_yaml("tardis_configv1_density_exponential_nebular.yml")

In [4]:
# Create the model
model = SimulationState.from_csvy(config, atom_data)


INFO:tardis.model.matter.decay:Decaying abundances for 2600540.0352 seconds
/Users/anirbandutta/Software/tardis/tardis/model/matter/composition.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '7.961942738176252e-23' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  isotope_mass_df[isotope_tuple] = isotope_mass


In [5]:
gamma_ray_lines = get_nuclear_lines_database(atom_data_file)

In [6]:
raw_isotope_abundance = model.composition.raw_isotope_abundance

In [7]:
# Construct the Plasma

input = [Density, Abundance, IsotopeAbundance, AtomicData, AtomicMass, IsotopeNumberDensity, NumberDensity, SelectedAtoms, IsotopeMass]

plasma = BasePlasma(plasma_properties=input, density = model.density, 
                    abundance=model.abundance, isotope_abundance=model.composition.raw_isotope_abundance,
                    atomic_data = atom_data)

In [8]:
shell_masses = model.volume * model.density

In [9]:
isotope_dict = create_isotope_dicts(raw_isotope_abundance, shell_masses)

In [10]:
inventories_dict = create_inventories_dict(isotope_dict)

In [11]:
total_decays = calculate_total_decays(inventories_dict, 80)

In [12]:
create_isotope_decay_df(total_decays, gamma_ray_lines)

number_of_decays decay_mode radiation  \
shell_number isotope                                          
0            Fe52         4.075312e+34         EC        bp   
             Fe52         4.075312e+34         EC     bp av   
             Fe52         4.075312e+34         EC         e   
             Fe52         4.075312e+34         EC         e   
             Fe52         4.075312e+34         EC         e   
...                                ...        ...       ...   
65           V48          0.000000e+00         EC         g   
             V48          0.000000e+00         EC         g   
             V48          0.000000e+00         EC         g   
             V48          0.000000e+00         EC         g   
             V48          0.000000e+00         EC         g   

                      radiation_energy_keV  radiation_intensity  \
shell_number isotope                                              
0            Fe52               340.000000              55.4900   
             Fe52               340.000001              55.4900   
             Fe52                 5.190000              26.9000   
             Fe52                 0.610000              62.5000   
             Fe52               162.149000               0.6990   
...                                    ...                  ...   
65           V48               1312.105000              98.2000   
             V48               1437.521000               0.1200   
             V48               2240.396000               2.3330   
             V48               2375.200000               0.0087   
             V48               2420.940000               0.0067   

                      energy_per_channel_keV  decay_energy_keV  
shell_number isotope                                            
0            Fe52                 188.666000      7.688728e+36  
             Fe52                 188.666000      7.688728e+36  
             Fe52                   1.396110      5.689584e+34  
             Fe52                   0.381250      1.553713e+34  
             Fe52                   1.133422      4.619046e+34  
...                                      ...               ...  
65           V48                 1288.487110      0.000000e+00  
             V48                    1.725025      0.000000e+00  
             V48                   52.268439      0.000000e+00  
             V48                    0.206642      0.000000e+00  
             V48                    0.162203      0.000000e+00  

[18744 rows x 7 columns]